### 패션 MNIST 
- class : 10종류
- 28*28=784
- sample 수 : 70000개

In [1]:
# 데이터 불러오기
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# 학습 데이터 불러오기
train_data = datasets.FashionMNIST(
                root = "./Data",
                train = True,
                download = True,
                transform = ToTensor() # 이미지를 텐서로 
)

100%|██████████| 26.4M/26.4M [00:01<00:00, 17.3MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 275kB/s]
100%|██████████| 4.42M/4.42M [00:00<00:00, 5.16MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 11.1MB/s]


In [3]:
# 테스트 데이터 불러오기
test_data = datasets.FashionMNIST(
                root = "./Data",
                train = False,
                download = True,
                transform = ToTensor() # 이미지를 텐서로 
)

In [4]:
# train_data의 data와 target의 크기
print(train_data.data.shape)
print(train_data.targets.shape)

torch.Size([60000, 28, 28])
torch.Size([60000])


In [5]:
# test_data의 data와 target의 크기
print(test_data.data.shape)
print(test_data.targets.shape)

torch.Size([10000, 28, 28])
torch.Size([10000])


In [6]:
# Train data를 훈련데이터와 검증데이터로 나누기
from sklearn.model_selection import train_test_split

# 채널 차원 추가 및 정규화 
train_input = train_data.data.unsqueeze(1).float() / 255.0 
train_target = train_data.targets

test_input = test_data.data.unsqueeze(1).float() / 255.0
test_target = test_data.targets

train_input, val_input, train_target, val_target = train_test_split(
                                                    train_input,
                                                    train_target,
                                                    test_size=0.2,
                                                    random_state=42
)

In [7]:
# Dimension 확인

print(train_input.shape, train_target.shape)
print(val_input.shape, val_target.shape)

torch.Size([48000, 1, 28, 28]) torch.Size([48000])
torch.Size([12000, 1, 28, 28]) torch.Size([12000])


---
### CNN 신경망 만들기

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [9]:
# Dataset 및 DataLoader 생성
batch_size = 32 # mini batch
train_dataset = TensorDataset(train_input, train_target)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = TensorDataset(val_input, val_target)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

### 모델 정의

In [10]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 7 * 7, 128) 
        # (conv2의 크기(64), 28pixle의 max pooling을 2번했으므로 28/2/2 = 7 * 7)
        self.relu3 = nn.ReLU()
        self.dropout = nn.Dropout(0.9)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)

        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [11]:
# 손실함수와 옵티마이저
model = CNNModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [12]:
# 학습 함수 정의
def train(model, train_loader, criterion, optimizer, device):
    model.train() # 뉴런들이 훈련모드로 동작하도록 설정 
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad() # 이전 반복에서 계산된 그래디언트를 초기화 
        outputs = model(inputs) # 내부적으로 model.forward(inputs) 호출하여 예측값 얻음 
        loss = criterion(outputs, targets) # 모델의 예측값과 실제 타겟 간의 손실을 계산 
        loss.backward() # 손실에 대한 그래디언트를 계산하고 역전파 
        optimizer.step() # 계산된 그래디언트를 사용하고 모델의 파라미터를 업데이트 
    return loss.item() # 현재 배치의 손실값을 반환 

In [13]:
# 평가함수 
def evaluate(model, val_loader, criterion, device):
    model.eval() # 평가모드로 동작하도록 설정 
    total_loss = 0 # 전체 손실 합계 
    correct = 0 # 정확하게 예측한 샘플 수 
    total = 0 # 전체 샘플 수
    with torch.no_grad() : # 평가 중에는 그래디언트 계산이 필요 없으므로 메모리 사용량 및 연산 속도 향상
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs) # 예측값 발생 
            loss = criterion(outputs, targets) # 손실계산
            total_loss += loss.item() # 전체 손실에 더하기 
            _, predicted = outputs.max(1) # 전체 샘플에 대해 가장 높은 확률을 가진 인덱스 
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item() # 예측값과 실제값이 일치하는 경우 True를 발생 
        return total_loss/len(val_loader), correct/total # 평균손실과 정확도 

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

cuda


CNNModel(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=3136, out_features=128, bias=True)
  (relu3): ReLU()
  (dropout): Dropout(p=0.9, inplace=False)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [15]:
# 훈련 반복
num_epochs = 50

for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    print(f"Epoch[{epoch+1}/{num_epochs}], Loss :{train_loss:.4f}")

Epoch[1/50], Loss :1.3029
Epoch[2/50], Loss :1.4533
Epoch[3/50], Loss :1.0074
Epoch[4/50], Loss :0.9139
Epoch[5/50], Loss :1.1451
Epoch[6/50], Loss :0.8083
Epoch[7/50], Loss :0.8098
Epoch[8/50], Loss :0.7533
Epoch[9/50], Loss :0.6325
Epoch[10/50], Loss :0.8627
Epoch[11/50], Loss :0.9262
Epoch[12/50], Loss :0.7938
Epoch[13/50], Loss :0.5361
Epoch[14/50], Loss :0.6431
Epoch[15/50], Loss :0.6117
Epoch[16/50], Loss :0.6636
Epoch[17/50], Loss :0.6970
Epoch[18/50], Loss :0.7375
Epoch[19/50], Loss :0.6616
Epoch[20/50], Loss :0.6713
Epoch[21/50], Loss :0.4591
Epoch[22/50], Loss :0.5644
Epoch[23/50], Loss :0.5130
Epoch[24/50], Loss :0.3833
Epoch[25/50], Loss :0.8066
Epoch[26/50], Loss :0.6793
Epoch[27/50], Loss :0.6696
Epoch[28/50], Loss :0.6022
Epoch[29/50], Loss :0.8106
Epoch[30/50], Loss :0.5470
Epoch[31/50], Loss :0.4196
Epoch[32/50], Loss :0.6113
Epoch[33/50], Loss :0.4654
Epoch[34/50], Loss :0.6212
Epoch[35/50], Loss :0.7881
Epoch[36/50], Loss :0.2861
Epoch[37/50], Loss :0.5938
Epoch[38/5

In [16]:
# 훈련 평가
train_loss, train_accuracy = evaluate(model, train_loader, criterion, device)
print(f"Loss : {train_loss:.4f}, Accuracy : {train_accuracy:.4f}")

Loss : 0.2035, Accuracy : 0.9316


In [17]:
# 검증 평가
val_loss, val_accuracy = evaluate(model, val_loader, criterion, device)
print(f"Loss : {val_loss:.4f}, Accuracy : {val_accuracy:.4f}")

Loss : 0.2947, Accuracy : 0.9053


In [18]:
# 일반화 평가
test_dataset = TensorDataset(test_input, test_target)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

test_loss, test_accuracy = evaluate(model, test_loader, criterion, device)
print(f"Loss : {test_loss:.4f}, Accuracy : {test_accuracy:.4f}")

Loss : 0.3408, Accuracy : 0.8984


---
### Image를 만들어서 예측해 보기
: train_data의 50번째로 이미지 만들어 저장하기

In [19]:
from PIL import Image
import numpy as np

In [27]:
abc = np.array(train_data.data[50]).reshape(28,28)
abc.shape

(28, 28)

In [29]:
image = Image.fromarray(abc)
image

In [30]:
# 저장하기
image.save("./Data/fashion_mnist_50.png")

---
### 이미지를 불러와서 예측해 보기

In [31]:
img = Image.open("./Data/fashion_mnist_50.png")
img

In [32]:
# img를 numpy 배열로 변환
img = np.array(img)
img.shape

(28, 28)

In [33]:
# numpy배열을 torch로 변환
img = torch.from_numpy(img)
img.shape

torch.Size([28, 28])

In [34]:
# 정규화 및 channel 차원 추가
img = img.unsqueeze(0).float() / 255.0
img.shape

torch.Size([1, 28, 28])

In [35]:
# class들의 이름 정의
classes = ['티셔츠','바지','스웨터','드레스','코트','샌달','셔츠','스니커즈','가방','앵글부츠']

In [39]:
# 예측 함수 
def predict_single_image(model, image, device, classes):
    model.eval()
    with torch.no_grad():
        image = image.to(device)
        output = model(image.unsqueeze(0))
        _, predicted = torch.max(output, 1) # 가장 높은 확률의 클래스 인덱스 선택 
        predicted_class = classes[predicted.item()]
    return predicted_class

In [40]:
predict_single_image(model, img, device, classes)

'드레스'